# Office

In [13]:
import pandas as pd

## Database connection

In [14]:
import yaml
from sqlalchemy import create_engine


with open("../config.yml", "r") as file:
	config = yaml.safe_load(file)
	config_OLTP = config["OLTP"]
	config_OLAP = config["OLAP"]


url_OLTP = (f"{config_OLTP['drivername']}://{config_OLTP['user']}:{config_OLTP['password']}"
			f"@{config_OLTP['host']}:{config_OLTP['port']}/{config_OLTP['database_name']}")

url_OLAP = (f"{config_OLAP['drivername']}://{config_OLAP['user']}:{config_OLAP['password']}"
			f"@{config_OLAP['host']}:{config_OLAP['port']}/{config_OLAP['database_name']}")

OLTP_connection = create_engine(url_OLTP)
OLAP_connection = create_engine(url_OLAP)

## Extraction

In [15]:
sede = pd.read_sql_table("sede", OLTP_connection)
city = pd.read_sql_table("ciudad", OLTP_connection)

In [16]:
sede.head(1)

,sede_id,nombre,direccion,telefono,nombre_contacto,ciudad_id,cliente_id
0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,1,4


In [17]:
city.head(1)

,ciudad_id,nombre,departamento_id
0,6,BUGA,1


## Transformation

In [18]:
office_dimension = sede.merge(city, on="ciudad_id", how="left")

In [19]:
office_dimension = office_dimension.rename(
	columns={
		"sede_id": "office_id",
		"nombre_x": "office_name",
		"nombre_y": "office_city",
		"departamento_id": "office_region",
	}
)

In [20]:
office_dimension = office_dimension[
	[
		"office_id",
		"office_name",
		"office_city",
		"office_region",
	]
]

In [21]:
office_dimension.head(2)

,office_id,office_name,office_city,office_region
0,10,FARALLONES /123,CALI,1
1,11,REMEDIOZ/ 123,CALI,1


## Load

In [22]:
office_dimension.to_sql(
	"OFFICE_DIMENSION", OLAP_connection, if_exists="replace", index=False
)

52